In [2]:
import os
from moralis import evm_api
import pandas as pd
import numpy as np
import time

from src.crypto_explorer.api import MoralisAPI

In [2]:
self = MoralisAPI(False, os.getenv("moralis_api_key"))


In [3]:
kwargs = {}
wallet = os.getenv("polygon_wallet")
address = "0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6"

In [4]:
transactions = pd.read_parquet(
    "api/tests/test_data/transactions.parquet"
)

In [5]:
rng = np.random.default_rng(42)
transactions['block_number'] = rng.integers(low=100_000, high=150_000, size=len(transactions))
transactions

,hash,nonce,transaction_index,from_address_entity,from_address_entity_logo,from_address,from_address_label,to_address_entity,to_address_entity_logo,to_address,...,block_number,block_hash,transaction_fee,method_label,nft_transfers,erc20_transfers,native_transfers,summary,possible_spam,category
0,dummy_hash,96,72,None,None,from_dummy_wallet,None,ParaSwap,https://entities-logos.s3.us-east-1.amazonaws....,to_dummy_wallet,...,104462,dummy_block_hash,0.032092,None,[],[{'address': '0xc2132d05d31c914a87c6611c10748a...,[],Swapped 28.52 USDT for 0.0003421 WBTC,False,token swap
1,dummy_hash,95,53,None,None,from_dummy_wallet,None,ParaSwap,https://entities-logos.s3.us-east-1.amazonaws....,to_dummy_wallet,...,138697,dummy_block_hash,0.046096,None,[],[{'address': '0xc2132d05d31c914a87c6611c10748a...,[],Swapped 28.52 USDT for 0.0003437 WBTC,False,token swap
2,dummy_hash,94,67,None,None,from_dummy_wallet,None,ParaSwap,https://entities-logos.s3.us-east-1.amazonaws....,to_dummy_wallet,...,132728,dummy_block_hash,0.064322,None,[],[{'address': '0xc2132d05d31c914a87c6611c10748a...,[],Swapped 28.52 USDT for 0.0003512 WBTC,False,token swap
3,dummy_hash,93,76,None,None,from_dummy_wallet,None,ParaSwap,https://entities-logos.s3.us-east-1.amazonaws....,to_dummy_wallet,...,121943,dummy_block_hash,0.013694,None,[],[{'address': '0xc2132d05d31c914a87c6611c10748a...,[],Swapped 0.0003246 WBTC for 27.98 USDT,False,token swap
4,dummy_hash,92,40,None,None,from_dummy_wallet,None,ParaSwap,https://entities-logos.s3.us-east-1.amazonaws....,to_dummy_wallet,...,121650,dummy_block_hash,0.019240,None,[],[{'address': '0xc2132d05d31c914a87c6611c10748a...,[],Swapped 0.0003243 WBTC for 28.25 USDT,False,token swap
5,dummy_hash,91,54,None,None,from_dummy_wallet,None,ParaSwap,https://entities-logos.s3.us-east-1.amazonaws....,to_dummy_wallet,...,142929,dummy_block_hash,0.018425,None,[],[{'address': '0x1bfd67037b42cf73acf2047067bd4f...,[],Swapped 0.0003254 WBTC for 29.34 USDT,False,token swap
6,dummy_hash,90,78,None,None,from_dummy_wallet,None,ParaSwap,https://entities-logos.s3.us-east-1.amazonaws....,to_dummy_wallet,...,104297,dummy_block_hash,0.014292,None,[],[{'address': '0x1bfd67037b42cf73acf2047067bd4f...,[],Swapped 27.44 USDT for 0.0003246 WBTC,False,token swap
7,dummy_hash,89,67,None,None,from_dummy_wallet,None,ParaSwap,https://entities-logos.s3.us-east-1.amazonaws....,to_dummy_wallet,...,134868,dummy_block_hash,0.008896,None,[],[{'address': '0x1bfd67037b42cf73acf2047067bd4f...,[],Swapped 27.44 USDT for 0.0003243 WBTC,False,token swap
8,dummy_hash,88,99,None,None,from_dummy_wallet,None,ParaSwap,https://entities-logos.s3.us-east-1.amazonaws....,to_dummy_wallet,...,110073,dummy_block_hash,0.012739,None,[],[{'address': '0x1bfd67037b42cf73acf2047067bd4f...,[],Swapped 27.44 USDT for 0.0003254 WBTC,False,token swap
9,dummy_hash,87,79,None,None,from_dummy_wallet,None,ParaSwap,https://entities-logos.s3.us-east-1.amazonaws....,to_dummy_wallet,...,104708,dummy_block_hash,0.022342,None,[],[{'address': '0x1bfd67037b42cf73acf2047067bd4f...,[],Swapped 0.0002958 WBTC for 25.99 USDT,False,token swap


In [ ]:
class Testclass:
    def __init__(self):
        self.api_client = MoralisAPI(verbose=False, api_key="dummy_key")

        self.transactions = pd.read_parquet(
            "api/tests/test_data/transactions.parquet"
        )

        self.aligned_transactions = [
            self.transactions.iloc[column].to_dict()
            for column in range(self.transactions.shape[0])
        ]

        self.dummy_balances = pd.read_parquet("dummy_balances.parquet")[
            0
        ].to_numpy()

        self.dummy_balances = [list(x) for x in self.dummy_balances]

    # Moralis API
    def fetch_transactions(
        self,
        wallet: str,
        excluded_categories: list | None = None,
        **kwargs,
    ) -> list:
        """
        Retrieves transaction history for the specified wallet address 
        while filtering out transactions that are marked as spam or 
        belong to any of the excluded categories.

        Parameters
        ----------
        wallet : str
            The wallet address for which to retrieve the transaction 
            history.
        excluded_categories : list or None, optional
            A list of transaction categories to exclude. If None, a 
            default list of categories including "contract interaction",
            "token receive", "airdrop", "receive", "approve", and "send"
            will be used.
        **kwargs : dict
            Additional keyword arguments to filter transactions, such 
            as:
                - **from_block**: int
                    The minimum block number to start retrieving 
                    transactions.
                - **to_block**: int
                    The maximum block number to stop retrieving 
                    transactions.
                - **from_date**: str
                    The start date 
                    (in seconds or a momentjs-compatible datestring).
                - **to_date**: str
                    The end date 
                    (in seconds or a momentjs-compatible datestring).
                - **include_internal_transactions**: bool
                    Whether to include internal transactions in the 
                    results.
                - **nft_metadata**: bool
                    Whether to include NFT metadata in the results.
                - **cursor**: str
                    A pagination cursor returned from previous 
                    responses.
                - **order**: str
                    The order of transactions, either "ASC" for 
                    ascending or "DESC" for descending.
                - **limit**: int
                    The maximum number of transactions to retrieve.

        Returns
        -------
        list
            A list of transaction dictionaries that have been filtered to exclude
            spam and the specified categories.

        Side Effects
        ------------
        Logs the start and completion of the transaction retrieval process.
        """
        transactions = self.transactions

        rng = np.random.default_rng(42)
        transactions["block_number"] = rng.integers(
            low=100_000, high=150_000, size=len(transactions)
        )

        aligned_transactions = [
            transactions.iloc[column].to_dict()
            for column in range(transactions.shape[0])
        ]

        return aligned_transactions

    # Moralis API
    def fetch_wallet_token_balances(
            self, 
            wallet_address: str, 
            block_number: int,
        ) -> pd.DataFrame:
        """
        Retrieves the token balances for a specified wallet address at
        a given block number.

        Parameters
        ----------
        wallet_address : str
            The wallet address for which to fetch token balances.
        block_number : int
            The block number at which to evaluate the wallet's token balances.

        Returns
        -------
        pandas.DataFrame
            A DataFrame indexed by token symbol with a single column
            (named after the block number) showing the balance of
            each token.
        """
        dummy_balances = []

        for balance in self.dummy_balances:
            dummy_balances.extend(balance)

        return dummy_balances

    # Moralis API
    def fetch_token_price(
        self,
        block_number: int,
        address: str = "0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6",
    ) -> pd.Series:
        """
        Retrieves the token price at a specified block number using the 
        Moralis API.

        Parameters
        ----------
        block_number : int
            The block number at which to fetch the token price.
        address : str
            The address of the token to retrieve the price for.

        Returns
        -------
        pandas.Series
            A Series containing the token price data as returned by the 
            Moralis API.
        """
        return {
            "usdPrice": 100_000,
            "blockTimestamp": 1741827878000,
        }

    # Moralis API
    def fetch_block(self, unix_date: int | str) -> pd.Series:
        """
        Retrieves block information corresponding to a given Unix 
        timestamp.

        Parameters
        ----------
        unix_date : int or str
            The Unix timestamp to retrieve the block information for.

        Returns
        -------
        dict
            A dictionary containing block information as returned by the
            Moralis API.
        """
        return {"block": 500_000}

    def get_wallet_token_balances(
        self,
        wallet_address: str,
        block_number: int,
    ) -> pd.DataFrame:
        result = self.fetch_wallet_token_balances(wallet_address, block_number)
        result_df = pd.DataFrame(result).dropna(subset="security_score")

        result_df["token_balance"] = result_df["balance"].astype(
            "int64"
        ) / 10 ** result_df["decimals"].astype("int64")

        inline_result = result_df[["symbol", "token_balance"]].set_index(
            "symbol"
        )

        inline_result.columns = [str(block_number)]
        return inline_result

    def get_wallet_token_balances_history(
        self, 
        wallet_address: str, 
        token_address: str,
        **kwargs: dict,
    ) -> pd.DataFrame:
        """
        Retrieves the historical token balances for a specific wallet
        and token.

        This method gathers all transactions for the given wallet
        address, extracts the block numbers, and then for each block
        (including the latest block), it queries the token balances and
        token price. The resulting data includes the token balance,
        corresponding USD price, and the block timestamp at which the
        price was retrieved.

        Parameters
        ----------
        wallet_address : str
            The wallet address for which to fetch the token balances
            history.
        token_address : str
            The address of the token to retrieve the price for at each
            block.
        **kwargs : dict
            Additional keyword arguments to filter transactions, such
            as:
                - **from_block**: int
                    The minimum block number to start retrieving
                    transactions.
                - **to_block**: int
                    The maximum block number to stop retrieving
                    transactions.
                - **from_date**: str
                    The start date
                    (in seconds or a momentjs-compatible datestring).
                - **to_date**: str
                    The end date
                    (in seconds or a momentjs-compatible datestring).
                - **include_internal_transactions**: bool
                    Whether to include internal transactions in the
                    results.
                - **nft_metadata**: bool
                    Whether to include NFT metadata in the results.
                - **cursor**: str
                    A pagination cursor returned from previous
                    responses.
                - **order**: str
                    The order of transactions, either "ASC" for
                    ascending or "DESC" for descending.
                - **limit**: int
                    The maximum number of transactions to retrieve.

        Returns
        -------
        pd.DataFrame
            A DataFrame containing the token balances (transposed),
            USD price, and block timestamp for each evaluated block.
        """
        transactions = self.fetch_transactions(wallet_address, **kwargs)

        block_numbers = (
            pd.DataFrame(transactions)['block_number']
            .astype(int)
            .tolist()
        )

        token_balances = []
        time_now = int(time.time())
        last_block = self.fetch_block(time_now)["block"]
        updated_blocks = [*block_numbers, last_block]

        for block in updated_blocks:

            temp_df = self.get_wallet_token_balances(wallet_address, block).T
            token_price = self.fetch_token_price(block, token_address)

            temp_df['usdPrice'] = token_price['usdPrice']
            temp_df['blockTimestamp'] = pd.Timestamp(
                int(token_price['blockTimestamp']),
                unit='ms',
            )

            token_balances.append(temp_df)

        return pd.concat(token_balances, axis=0)

In [24]:
transactions = self.api_client.fetch_transactions(wallet_address, **kwargs)

block_numbers = (
    pd.DataFrame(transactions)['block_number']
    .astype(int)
    .tolist()
)

token_balances = []
time_now = int(time.time())
last_block = self.api_client.fetch_block(time_now)["block"]
updated_blocks = [*block_numbers, last_block]

for block in updated_blocks:

    temp_df = self.api_client.get_wallet_token_balances(wallet_address, block).T
    token_price = self.api_client.fetch_token_price(block, token_address)

    temp_df['usdPrice'] = token_price['usdPrice']
    temp_df['blockTimestamp'] = pd.Timestamp(
        int(token_price['blockTimestamp']),
        unit='ms',
    )

    token_balances.append(temp_df)

pd.concat(token_balances, axis=0)

symbol,WBTC,USDT,WBTC,USDT,WBTC,USDT,WBTC,USDT,WBTC,USDT,...,WBTC,USDT,WBTC,USDT,WBTC,USDT,WBTC,USDT,usdPrice,blockTimestamp
104462,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
138697,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
132728,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
121943,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
121650,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
142929,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
104297,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
134868,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
110073,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
104708,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38


In [29]:
mapi.fetch_wallet_token_balances()

NameError: name 'mapi' is not defined

In [27]:
self.api_client.get_wallet_token_balances(wallet_address, block)

,500000
symbol,
WBTC,5.884305
USDT,326940.373369
WBTC,2.050381
USDT,148.035889
WBTC,8.218896
USDT,62523.502209
WBTC,8.438703
USDT,2565.726409
WBTC,5.710890


In [25]:
self.api_client.fetch_token_price(block, token_address)

{'usdPrice': 100000, 'blockTimestamp': 1741827878000}

In [17]:
self.api_client = Testclass()

In [18]:
result = self.api_client.get_wallet_token_balances_history(
    wallet_address="0x1", token_address="0x2"
)
result

symbol,WBTC,USDT,WBTC,USDT,WBTC,USDT,WBTC,USDT,WBTC,USDT,...,WBTC,USDT,WBTC,USDT,WBTC,USDT,WBTC,USDT,usdPrice,blockTimestamp
104462,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
138697,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
132728,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
121943,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
121650,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
142929,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
104297,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
134868,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
110073,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38
104708,5.884305,326940.373369,2.050381,148.035889,8.218896,62523.502209,8.438703,2565.726409,5.71089,1544.804416,...,9.808711,3010.936384,1.660195,243087.455521,9.496194,113.379904,6.045387,15625.0,100000,2025-03-13 01:04:38


In [30]:
kwargs = {}
mAPI = MoralisAPI(True, os.getenv("moralis_api_key"))

wallet = os.getenv("polygon_wallet")
address = "0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6"

In [33]:
mAPI.fetch_wallet_token_balances(wallet, 69_000_000)

[{'token_address': '0x0b546a51e8a872c43e3f5f57fd60871286299ff6',
  'symbol': 'Claim $TRUMP ( t.ly/TRUMP ) - #1',
  'name': '$TRUMP AIRDROP',
  'logo': None,
  'thumbnail': None,
  'decimals': 0,
  'balance': '1',
  'possible_spam': False,
  'verified_contract': False,
  'total_supply': '1',
  'total_supply_formatted': '1',
  'percentage_relative_to_total_supply': 100,
  'security_score': None},
 {'token_address': '0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6',
  'symbol': 'WBTC',
  'name': '(PoS) Wrapped BTC',
  'logo': 'https://logo.moralis.io/0x89_0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6_093b6724c111304586325c9abb31f068.png',
  'thumbnail': 'https://logo.moralis.io/0x89_0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6_093b6724c111304586325c9abb31f068.png',
  'decimals': 8,
  'balance': '103712',
  'possible_spam': False,
  'verified_contract': True,
  'total_supply': '321369810944',
  'total_supply_formatted': '3213.69810944',
  'percentage_relative_to_total_supply': 3.2271855186196e-05,
 

In [20]:
wallet_address = wallet
token_address = address

transactions = mAPI.fetch_transactions(wallet_address, **kwargs)

block_numbers = (
    pd.DataFrame(transactions)['block_number']
    .astype(int)
    .tolist()
)

token_balances = []
time_now = int(time.time())
last_block = mAPI.fetch_block(time_now)["block"]
updated_blocks = [*block_numbers, last_block]

for block in updated_blocks[:2]:
    mAPI.logger.info(f"Getting token balances for block {block}.")

    temp_df = mAPI.get_wallet_token_balances(wallet_address, block).T
    token_price = mAPI.fetch_token_price(block, token_address)

    temp_df['usdPrice'] = token_price['usdPrice']
    temp_df['blockTimestamp'] = pd.Timestamp(
        int(token_price['blockTimestamp']),
        unit='ms',
    )

    token_balances.append(temp_df)

    progress = len(token_balances) / len(updated_blocks)
    progress_abs = f"{len(token_balances)} / {len(updated_blocks)}"

    mAPI.logger.info(f"Progress: {progress:.2%} - {progress_abs}")

pd.concat(token_balances)

INFO 22:35:30: Retrieving transactions for wallet: 0xC29729D92b9D6bb5EE2bE9dFad1E6C2A2E5EE39f
INFO 22:35:32: Retrieved 61 transactions
INFO 22:35:33: Getting token balances for block 68978933.
INFO 22:35:34: Progress: 1.61% - 1 / 62
INFO 22:35:34: Getting token balances for block 68939048.
INFO 22:35:36: Progress: 3.23% - 2 / 62


symbol,WBTC,usdPrice,blockTimestamp,USDT
68978933,0.001037,83198.726630,2025-03-13 01:04:38,NaN
68939048,0.000695,82863.586914,2025-03-12 01:21:02,28.526519


In [22]:
token_price['usdPrice']

82863.58691394927

In [35]:
dummy_balances = pd.read_parquet("dummy_balances.parquet")[
    0
].to_numpy()

dummy_balances = [list(x) for x in dummy_balances]

In [38]:
dummy_balances[1]

[{'balance': '7',
  'decimals': 0,
  'logo': None,
  'name': '$SPAM AIRDROP',
  'percentage_relative_to_total_supply': 100.0,
  'possible_spam': False,
  'security_score': None,
  'symbol': 'Claim $SPAM ( t.ly/SPAM ) - #1',
  'thumbnail': None,
  'token_address': 'dummy_spam_token',
  'total_supply': '7',
  'total_supply_formatted': '7',
  'verified_contract': False},
 {'balance': '205038070',
  'decimals': 8,
  'logo': 'https://logo.moralis.io/0x89_0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6_093b6724c111304586325c9abb31f068.png',
  'name': '(PoS) Wrapped BTC',
  'percentage_relative_to_total_supply': 0.06379133921893343,
  'possible_spam': False,
  'security_score': 85.0,
  'symbol': 'WBTC',
  'thumbnail': 'https://logo.moralis.io/0x89_0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6_093b6724c111304586325c9abb31f068.png',
  'token_address': '0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6',
  'total_supply': '321419917673',
  'total_supply_formatted': '3214.19917673',
  'verified_contract': True}

In [39]:
        expected_result = pd.DataFrame(
            [
                {
                    "WBTC": 5.884305,
                    "USDT": 326940.373369,
                    "usdPrice": 100000,
                    "blockTimestamp": pd.Timestamp("2025-03-13 01:04:38"),
                },
                {
                    "WBTC": 2.050381,
                    "USDT": 148.035889,
                    "usdPrice": 100000,
                    "blockTimestamp": pd.Timestamp("2025-03-13 01:04:38"),
                },
                {
                    "WBTC": 8.218896,
                    "USDT": 62523.502209,
                    "usdPrice": 100000,
                    "blockTimestamp": pd.Timestamp("2025-03-13 01:04:38"),
                },
                {
                    "WBTC": 8.438703,
                    "USDT": 2565.726409,
                    "usdPrice": 100000,
                    "blockTimestamp": pd.Timestamp("2025-03-13 01:04:38"),
                },
                {
                    "WBTC": 5.710890,
                    "USDT": 1544.804416,
                    "usdPrice": 100000,
                    "blockTimestamp": pd.Timestamp("2025-03-13 01:04:38"),
                },
                {
                    "WBTC": 5.182523,
                    "USDT": 4096.000000,
                    "usdPrice": 100000,
                    "blockTimestamp": pd.Timestamp("2025-03-13 01:04:38"),
                },
                {
                    "WBTC": 9.808711,
                    "USDT": 3010.936384,
                    "usdPrice": 100000,
                    "blockTimestamp": pd.Timestamp("2025-03-13 01:04:38"),
                },
                {
                    "WBTC": 1.660195,
                    "USDT": 243087.455521,
                    "usdPrice": 100000,
                    "blockTimestamp": pd.Timestamp("2025-03-13 01:04:38"),
                },
                {
                    "WBTC": 9.496194,
                    "USDT": 113.379904,
                    "usdPrice": 100000,
                    "blockTimestamp": pd.Timestamp("2025-03-13 01:04:38"),
                },
                {
                    "WBTC": 6.045387,
                    "USDT": 15625.000000,
                    "usdPrice": 100000,
                    "blockTimestamp": pd.Timestamp("2025-03-13 01:04:38"),
                },
                {
                    "WBTC": 5.884305,
                    "USDT": 326940.373369,
                    "usdPrice": 100000,
                    "blockTimestamp": pd.Timestamp("2025-03-13 01:04:38"),
                },
            ],
            pd.Index(
                [
                    "104462",
                    "138697",
                    "132728",
                    "121943",
                    "121650",
                    "142929",
                    "104297",
                    "134868",
                    "110073",
                    "104708",
                    "500000",
                ],
                name="symbol",
            ),
        )

In [44]:
expected_result.to_parquet("get_wallet_token_balances_history.parquet")

In [49]:
pd.read_parquet(
    r"api\tests\test_data\get_wallet_token_balances_history.parquet"
)

,WBTC,USDT,usdPrice,blockTimestamp
symbol,,,,
104462,5.884305,326940.373369,100000,2025-03-13 01:04:38
138697,2.050381,148.035889,100000,2025-03-13 01:04:38
132728,8.218896,62523.502209,100000,2025-03-13 01:04:38
121943,8.438703,2565.726409,100000,2025-03-13 01:04:38
121650,5.710890,1544.804416,100000,2025-03-13 01:04:38
142929,5.182523,4096.000000,100000,2025-03-13 01:04:38
104297,9.808711,3010.936384,100000,2025-03-13 01:04:38
134868,1.660195,243087.455521,100000,2025-03-13 01:04:38
110073,9.496194,113.379904,100000,2025-03-13 01:04:38


In [54]:
int(token_price.loc["blockTimestamp"])

1741742462000

In [59]:
pd.Series([100_000, 1741827878000], index=["usdPrice", "blockTimestamp"])

usdPrice                 100000
blockTimestamp    1741827878000
dtype: int64

In [ ]:
token_price.loc['usdPrice']

tokenName                                                    (PoS) Wrapped BTC
tokenSymbol                                                               WBTC
tokenLogo                    https://logo.moralis.io/0x89_0x1bfd67037b42cf7...
tokenDecimals                                                                8
nativePrice                  {'value': '384699029950015044829120', 'decimal...
usdPrice                                                          82863.586914
usdPriceFormatted                                     82863.586913949264656390
exchangeName                                                        Uniswap v3
exchangeAddress                     0x1F98431c8aD98523631AE4a59f267346ea31F984
tokenAddress                        0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6
toBlock                                                               68939048
priceLastChangedAtBlock                                               68938988
blockTimestamp                                      

In [16]:
pd.DataFrame(mAPI.fetch_wallet_token_balances(wallet_address, block))

,token_address,symbol,name,logo,thumbnail,decimals,balance,possible_spam,verified_contract,total_supply,total_supply_formatted,percentage_relative_to_total_supply,security_score
0,0x0b546a51e8a872c43e3f5f57fd60871286299ff6,Claim $TRUMP ( t.ly/TRUMP ) - #1,$TRUMP AIRDROP,None,None,0,1,False,False,1,1,1.000000e+02,NaN
1,0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6,WBTC,(PoS) Wrapped BTC,https://logo.moralis.io/0x89_0x1bfd67037b42cf7...,https://logo.moralis.io/0x89_0x1bfd67037b42cf7...,8,69502,False,True,321369810944,3213.69810944,2.162680e-05,85.0
2,0x2d52d7bb45f3bd3d22b3f1ff4992a013e2c9eb80,deBridge Airdrop https://t.ly/ethers,deBridge Airdrop https://t.ly/ethers,None,None,0,1,False,False,1,1,1.000000e+02,NaN
3,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,USDT,(PoS) Tether USD,https://logo.moralis.io/0x89_0xc2132d05d31c914...,https://logo.moralis.io/0x89_0xc2132d05d31c914...,6,28526519,False,True,848220833463791,848220833.463791,3.363100e-06,85.0
4,0xeb6a7b4772f1b0e94d35fb5c52117e00c3bd4a6e,Airdrop(s) to be claimed at https://t.ly/drops,,None,None,18,2000000000000000000,False,False,71111111111111111111111040000000000,71111111111111111.11111104,2.813000e-15,NaN
5,0xee51cf4ba627496a28f35012d539c048392fea2e,https://t.ly/ethers,deBridge Airdrop,None,None,0,1,False,False,1,1,1.000000e+02,NaN


In [ ]:
extend_data = []
for balance in Testclass().dummy_balances:
    extend_data.extend(balance)

In [17]:
mAPI.fetch_wallet_token_balances(wallet_address, block)

[{'token_address': '0x0b546a51e8a872c43e3f5f57fd60871286299ff6',
  'symbol': 'Claim $TRUMP ( t.ly/TRUMP ) - #1',
  'name': '$TRUMP AIRDROP',
  'logo': None,
  'thumbnail': None,
  'decimals': 0,
  'balance': '1',
  'possible_spam': False,
  'verified_contract': False,
  'total_supply': '1',
  'total_supply_formatted': '1',
  'percentage_relative_to_total_supply': 100,
  'security_score': None},
 {'token_address': '0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6',
  'symbol': 'WBTC',
  'name': '(PoS) Wrapped BTC',
  'logo': 'https://logo.moralis.io/0x89_0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6_093b6724c111304586325c9abb31f068.png',
  'thumbnail': 'https://logo.moralis.io/0x89_0x1bfd67037b42cf73acf2047067bd4f2c47d9bfd6_093b6724c111304586325c9abb31f068.png',
  'decimals': 8,
  'balance': '69502',
  'possible_spam': False,
  'verified_contract': True,
  'total_supply': '321369810944',
  'total_supply_formatted': '3213.69810944',
  'percentage_relative_to_total_supply': 2.1626798047969e-05,
  